In [ ]:
import re
import json
import os
import glob
import cv2
import paddle
from paddleocr import PaddleOCR
import matplotlib.pyplot as plt
import wandb
import numpy as np
import cv2

# put your wanDB token in prompt!
wandb.login()

In [ ]:
# flow

from Model import Preprocesser_utils

### 사용 예시 ###
#config
video_path = '/content/drive/MyDrive/Colab Notebooks/OCR-project/dataset/Lost Ark 2min.mp4' # 사용경로
save_folder1 = '/content/drive/MyDrive/Colab Notebooks/OCR-project/dataset/test/yellow'             # 노랑 저장경로
save_folder2 = '/content/drive/MyDrive/Colab Notebooks/OCR-project/dataset/test/white'             # 하양 저장경로
save_name_prefix = 'frame'                                                                  # unique 한 이름 보통 (닉네임 또는 직업명)
fps = 12                                                                                    # 원하는 프레임

# quality 파라미터 도 있다. => 700 ~ 850, type 2는 비율
# 만약 전처리 되는 사진들까지 저장하고 싶다면, save=True


video_preprocessor = VideoPreprocessor(video_path, save_folder1, save_name_prefix, fps=fps, size_type=1, quality=850, save=False)
video_preprocessor.preprocess_video(operations=['apply_mask', 'resize_image', 'extract_yellow','binarize_image', 'gaussian_blur'])

video_preprocessor = VideoPreprocessor(video_path, save_folder2, save_name_prefix, fps=fps, size_type=1, quality=850, save=False)
video_preprocessor.preprocess_video(operations=['apply_mask', 'resize_image', 'extract_white','binarize_image', 'gaussian_blur'])

## 아웃풋 -> blade_56.jpg

"""전처리 끝"""

In [ ]:
from Model import OCRModel_WDB

# 모델 객체 생성
ocr_model = OCRModel_DB(lang='korean', use_gpu=True, run_name='Try')    # OCRModel.py 와 달리 
                                                                        # run_name 을 설정해줘야한다.

# 이미지 경로 리스트
yellow_folder_path = save_folder1  # 노랑
white_folder_path = save_folder2   # 하양
gt_path = r'.json'          # label json 경로
output_path = r'.json'      # result(ouput).json 경로
error_path = r''

results = ocr_model.process_images(yellow_folder_path, white_folder_path, confidence_threshold=0.9)
ocr_model.save_results(results, output_path)

# 총 데미지 계산
print()
total_critical_damage, total_normal_damage, total_damage = ocr_model.calculate_total_damage(results)
print(f"Total Critical Damage: {total_critical_damage}")
print(f"Total Normal Damage: {total_normal_damage}")
print(f"Total Damage: {total_damage}")

print()
percent = ocr_model.evaluate_damage(total_damage, gt_path)
print(f"Damage Percent : {percent:.2f}\n")

# 성능 평가
critical_precision, critical_recall, critical_accuracy, normal_precision, normal_recall, normal_accuracy, total_accuracy = ocr_model.evaluate(output_path, gt_path)
print(f"Critical Precision: {critical_precision:.4f}")
print(f"Critical Recall: {critical_recall:.4f}")
print(f"Critical Accuracy: {critical_accuracy:.4f}\n")
print(f"Normal Precision: {normal_precision:.4f}")
print(f"Normal Recall: {normal_recall:.4f}")
print(f"Normal Accuracy: {normal_accuracy:.4f}\n")
print(f"Total Accuracy: {total_accuracy:.4f}")

print()
ocr_model.visualize_errors(output_path, gt_path, yellow_folder_path, white_folder_path, error_path)